In [13]:
import subprocess

def my_system(cmd):
    return subprocess.run(cmd, capture_output=True).stdout.decode("utf-8").split("\n")

def find(pattern, including=None):
    a = my_system(["find", ".", "-name", pattern])
    a = [l for l in a if len(l) != 0]
    if including == None:
        return a
    else: 
        res = []
        for l in a:
            if subprocess.run(["grep", "-q", including, l]).returncode == 0:
                res.append(l)
        return res

def rename(fromf, tof):
    subprocess.run(["git", "mv", fromf, tof])

def replace(fn, froms, tos):
    cmd = f"s/{froms}/{tos}/g"
    subprocess.run(["sed", "-i", "", cmd, fn])

def nkf(fn):
    subprocess.run(["nkf", "--in-place", fn])

In [92]:
find("*.html", including="php")

['./2015/CFP.en.html',
 './2015/program_JHPCN.en.html',
 './2015/invited_session.ja.html',
 './2015/index.en.html',
 './2015/program.en.html',
 './2015/program_JHPCN.ja.html',
 './2015/cfsponsor.ja.html',
 './2015/invited_session.en.html',
 './2015/include/invited_session.ja.html',
 './2015/include/schedule-cfp-ja.html',
 './2015/CFP.ja.html',
 './2015/program.ja.html',
 './2015/index.ja.html',
 './2015/QA.ja.html',
 './2015/program_test.ja.html',
 './2015/contact.ja.html',
 './2015/committee.en.html',
 './2015/program_photonics.ja.html',
 './2015/exhibit.ja.html',
 './2015/cfposters.en.html',
 './2015/committee.ja.html',
 './2015/program_photonics.en.html',
 './2015/exhibit_test.html',
 './2015/contact.en.html',
 './2015/cfposters.ja.html',
 './2015/preliminary_CFP.ja.html',
 './acsi-html-next/2015/CFP.en.html',
 './acsi-html-next/2015/program_JHPCN.en.html',
 './acsi-html-next/2015/invited_session.ja.html',
 './acsi-html-next/2015/index.en.html',
 './acsi-html-next/2015/program.en.ht

In [26]:
replace("2015/test2.html", "EUC-JP", "UTF8")

In [28]:
fn = "2015/test.html"
nkf(fn)
replace(fn, "EUC-JP", "UTF8")
replace(fn, ".html.ja", "ja.html")

In [29]:
for fn in find("*.html.ja"):
    if len(fn) == 0: 
        continue
    nkf(fn)
    replace(fn, "EUC-JP", "UTF8")
    replace(fn, ".html.ja", ".ja.html")
    replace(fn, ".html.en", ".en.html")
    rename(fn, fn.replace(".html.ja", ".ja.html"))
        

In [69]:
for fn in find("*.ja.html"):
    if len(fn) == 0: 
        continue
    nkf(fn)
    replace(fn, "EUC-JP", "UTF8")
    replace(fn, ".html.ja", ".ja.html")
    replace(fn, ".html.en", ".en.html")

In [30]:
for fn in find("*.html.en"):
    if len(fn) == 0: 
        continue
    nkf(fn)
    replace(fn, "EUC-JP", "UTF8")
    replace(fn, ".html.ja", ".ja.html")
    replace(fn, ".html.en", ".en.html")
    rename(fn, fn.replace(".html.en", ".en.html"))

In [31]:
for fn in find("*.ja.html"):
    if len(fn) == 0: 
        continue
    replace(fn, "ja.html", ".ja.html")
    replace(fn, "en.html", ".en.html")


In [75]:
for fn in find("*.html"):
    if len(fn) == 0: 
        continue
    nkf(fn)
    replace(fn, "EUC-JP", "UTF8")
    replace(fn, ".html.ja", ".ja.html")
    replace(fn, ".html.en", ".en.html")    

In [62]:
for fn in find("*.html"):
    if len(fn) == 0: 
        continue
    replace(fn, "-.ja.html", "-ja.html")


In [99]:
from pathlib import Path
import re
pat = re.compile(r"""<\?php include\("(.*)"\);\?>""")


def process_one_line(parent_dir, line):
    m = pat.search(line)
    if m:
        path = parent_dir / m[1]
        a = phpInclude(path)
        return ["<!-- included from " + m[1] + " Start -->\n"] + a + ["<!-- included from " + m[1] + " END -->\n"] 
    else:
        return [line]


def phpInclude(fn):
    '''returns array of lines with \n'''
    p_dir = Path(fn).parent
    try:
        with open(fn) as f:
            res = []
            for line in f.readlines():
                res += process_one_line(p_dir, line)
        return res
    except FileNotFoundError:
        print(f"cannot find {fn}, ignore")
        return []

def dump(filename, array):
    with open(filename, "w") as f:
        for line in array:
            f.write(line)

def include_and_replace(fn):
    res = phpInclude(fn)
    rep_fn = fn+".replaced"
    dump(rep_fn, res)
    subprocess.run(["mv", rep_fn, fn])

        

In [60]:
import re
pat = re.compile(r"""<\?php include\("(.*)"\);\?>""")

m = pat.search("""    <?php include("include/running-header.html");?>\n""")
print(m[1])

include/running-header.html


In [100]:
for fn in find("*.html", including="php"):
    print(fn)
    include_and_replace(fn)

./2015/program.en.html
cannot find 2015/include/program.html, ignore
./2015/program_JHPCN.ja.html
./2015/cfsponsor.ja.html
./2015/invited_session.en.html
./2015/include/invited_session.ja.html
./2015/include/schedule-cfp-ja.html
cannot find 2015/include/include/acsi-schedule-ja.html, ignore
./2015/CFP.ja.html
./2015/program.ja.html
cannot find 2015/include/program.html, ignore
./2015/index.ja.html
./2015/QA.ja.html
./2015/program_test.ja.html
cannot find 2015/include/program.html, ignore
./2015/contact.ja.html
./2015/committee.en.html
./2015/program_photonics.ja.html
./2015/exhibit.ja.html
./2015/cfposters.en.html
./2015/committee.ja.html
./2015/program_photonics.en.html
./2015/exhibit_test.html
./2015/contact.en.html
./2015/cfposters.ja.html
./2015/preliminary_CFP.ja.html
./acsi-html-next/2015/CFP.en.html
./acsi-html-next/2015/program_JHPCN.en.html
./acsi-html-next/2015/invited_session.ja.html
./acsi-html-next/2015/index.en.html
./acsi-html-next/2015/program.en.html
cannot find acsi-h

In [77]:
dump("test.html", res)